<a href="https://colab.research.google.com/github/st24hour/tutorial/blob/master/Neural_Style_Transfer_with_Eager_Execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Style Transfer with tf.keras


## Overview

이 튜토리얼에서 우리는 딥러닝을 사용하여 다른 이미지의 스타일로 이미지를 구성하는 방법을 배우게됩니다 (피카소나 반 고흐처럼 그릴 수 있기를 바란 적 있습니까?). 이것은  **neural style transfer**라고 알려져 있습니다.  이것은 Leon A. Gatys의 논문 인 [A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576)에 설명되어 있으며 반드시 읽어 봐야합니다.

그런데, neural style transfer가 무엇일까요?

 Neural style transfer는 3 가지 이미지, **콘텐츠** 이미지, **스타일 참조** 이미지 (유명한 화가의 작품 등) 및 원하는 **입력 이미지** 를 사용하는 최적화 기술입니다: 입력 이미지가 콘텐츠 이미지처럼 보이도록 변형되지만 스타일 이미지의 스타일처럼 "색칠"되도록 서로 섞습니다.

예를 들어, 이 거북이와 Katsushika Hokusai의 이미지 *The Great Wave off Kanagawa*를 봅시다.

<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/Green_Sea_Turtle_grazing_seagrass.jpg?raw=1" alt="Drawing" style="width: 200px;"/>
<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/The_Great_Wave_off_Kanagawa.jpg?raw=1" alt="Drawing" style="width: 200px;"/>

[Image of Green Sea Turtle](https://commons.wikimedia.org/wiki/File:Green_Sea_Turtle_grazing_seagrass.jpg)
-By P.Lindgren [CC BY-SA 3.0  (https://creativecommons.org/licenses/by-sa/3.0)], from Wikimedia Common


Hokusai가 거북이의 그림을 자신의 스타일로 그리기로 결정했다면 어떻게 될까요? 이와 같을까요?

<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/wave_turtle.png?raw=1" alt="Drawing" style="width: 500px;"/>

Neural style transfer는 신경 네트워크의 기능과 내부 표현을 보여주는 재미있고 흥미로운 기술입니다.
 

Neural style transfer의 원리는 두 이미지의 내용이 얼마나 다른지, $L_{content}$, 두 이미지의 스타일이 얼마나 다른지, $L_{ style}$를 표현하는 두 거리 함수를 정의하는 것입니다. 그 다음, 3 개의 이미지, 원하는 스타일 이미지, 원하는 컨텐츠 이미지 및 입력 이미지 (컨텐츠 이미지로 초기화 됨)가 주어지면 입력 이미지를 콘텐츠 이미지와 콘텐츠 거리가 최소화 되도록,  스타일 이미지와 스타일 거리가 최소화 되도록 변환합니다. 요약하면 기본 입력 이미지, 일치시킬 콘텐츠 이미지 및 일치시키고자하는 스타일 이미지를 사용합니다. 우리는 backpropagation으로 컨텐츠 및 스타일 거리 (losses)를 최소화하고 컨텐츠 이미지의 컨텐츠 및 스타일 이미지의 스타일과 일치하는 이미지를 만듭니다.

### 다루게 될 개념들:
이 튜토리얼에서 우리는 실제 경험을 쌓고 다음 개념을 중심으로 실습할 것입니다.

* **Eager Execution** - Operation을 즉각적으로 평가하는 TensorFlow의 imperative programming 환경 사용 
  * [Learn more about eager execution](https://www.tensorflow.org/programmers_guide/eager)
  * [See it in action](https://www.tensorflow.org/get_started/eager)
* **모델 정의를 위해 [Functional API](https://keras.io/getting-started/functional-api-guide/) 사용** - Functional API를 사용하여 필요한 중간 activations에 대한 액세스를 제공 할 모델을 만들 것입니다.
* **Pretrained model의 feature를 활용** - Pretrained된 모델과 그 feature map을 사용하는 방법을 배웁니다.
* **Custom training loops 구현** - 입력 parameter와 관련된 주어진 손실을 최소화하기 위해 optimizer를 설정하는 방법을 살펴 보겠습니다.

### Style transfer의 일반적인 단계들  :

1. Visualize data
2. Basic Preprocessing/preparing our data
3. Set up loss functions 
4. Create model
5. Optimize for loss function

## Setup

### Download Images

In [0]:
import os
img_dir = '/tmp/nst'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/d/d7/Green_Sea_Turtle_grazing_seagrass.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg

### Import and configure modules

In [0]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools

In [0]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

from tensorflow.python.keras.preprocessing import image as kp_image
from tensorflow.python.keras import models 
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

우리는 [eager execution](https://www.tensorflow.org/guide/eager)을 가능하게 하는 것으로 시작할 것입니다. Eager execution은 우리가 가장 명확하고 가장 판독 가능한 방식으로 작업할 수 있게 해줍니다.

In [0]:
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

In [0]:
# Set up some global values here
content_path = '/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg'
style_path = '/tmp/nst/The_Great_Wave_off_Kanagawa.jpg'

## Visualize the input

In [0]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img

In [0]:
def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)

이들은 콘텐츠 및 스타일 입력 이미지입니다. 우리는 콘텐츠 이미지의 콘텐츠로 이미지를 "생성"하지만 스타일 이미지의 스타일을 사용하기를 바랍니다.

In [0]:
plt.figure(figsize=(10,10))

content = load_img(content_path).astype('uint8')
style = load_img(style_path).astype('uint8')

plt.subplot(1, 2, 1)
imshow(content, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style, 'Style Image')
plt.show()

## Prepare the data
이미지를 쉽게 로드하고 사전 처리 할 수있는 메소드를 만들어 봅시다. 우리는 VGG 학습 과정과 동일한 전처리 과정을 수행합니다. VGG 네트워크는 각 채널이 `mean = [103.939, 116.779, 123.68]` 및 채널 BGR로 normalize 된 이미지로 학습됩니다.

In [0]:
def load_and_process_img(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

최적화의 결과를 보기 위해서 우리는 역 사전 처리 단계를 수행해야합니다. 또한 최적화 된 이미지는  $- \infty$에서  $- \infty$ 사이의 값을 가질 수 있으므로 0-255 범위에서 값을 유지하려면 clip해야합니다.

In [0]:
def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessiing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

### Define content and style representations
이미지의 콘텐츠 표현과 스타일 표현을 모두 얻으려면 우리 모델에서 중간 layer를 살펴볼 것입니다. 이러한 중간 layer는 고차원 feature를 나타냅니다. 우리는 이미지 분류에서 pretrained된 VGG19 네트워크를 사용할 것입니다. 이러한 중간 layer는 이미지의 콘텐츠 및 스타일 표현을 정의하는 데 필요합니다. 입력 이미지를 이러한 중간 layer에서 해당 스타일 및 내용 타겟 표현과 일치 시키도록 할 것입니다.


#### Why intermediate layers?

미리 학습된 이미지 분류 네트워크에서 이러한 중간 결과물로 스타일과 컨텐츠 표현을 정의 할 수 있는 이유가 궁금 할 수 있습니다. High level에서 이 현상은 네트워크가 이미지 분류를 수행하기 위해 (네트워크가 수행하도록 훈련 된) 이미지를 이해해야한다는 사실로 설명 할 수 있습니다. 여기에는 raw 이미지를 입력으로 사용하여 raw 이미지 내에있는 복잡한 피쳐를 이해하는 것으로 바뀌는 변환을 통해 내부 표현을 만드는 작업이 포함됩니다. 이것은 컨벌루션 뉴럴 네트워크가 잘 일반화 될 수있는 이유 중 일부입니다. 배경 노이즈 및 기타 배경과 같은 부분에 상관없이 클래스 (예 : 고양이 vs. 개) 내에서 불변하는 특징을 정의 할 수 있습니다. 따라서 원본 이미지가 입력되고 분류 레이블이 출력되는 곳 사이의 어떤 곳에서 모델은 복잡한 피쳐 추출기로 사용됩니다. 따라서 중간 레이어에 액세스하여 입력 이미지의 내용과 스타일을 설명 할 수 있습니다.

특히 네트워크에서 다음과 같은 중간 계층을 가져옵니다.


참고: VGG19 architecture

<img src="https://www.researchgate.net/profile/Clifford_Yang/publication/325137356/figure/fig2/AS:670371271413777@1536840374533/llustration-of-the-network-architecture-of-VGG-19-model-conv-means-convolution-FC-means.jpg" alt="Drawing" style="width: 200px;"/>

In [0]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer we are interested in
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

## Build the Model 
우리는 [VGG19](https://keras.io/applications/#vgg19)를 불러오고 우리의 입력 tensor를 모델에 입력으로 줄 것입니다. 이것은 콘텐츠, 스타일, 그리고 생성하는 이미지의 feature map (내용 및 스타일 표현)을 추출할 수 있도록 해줍니다.

원래의 논문에서와 같이 VGG19 네트워크를 이용할 것입니다. 또한 VGG19는 ResNet, Inception 등과 비교하면 비교적 단순한 모델이므로 feature map이 실제로 스타일 이전에 더 효과적입니다.

우리의 스타일 및 콘텐츠 feature map에 해당하는 중간 layer의 출력을 얻을 것이며, Keras [**Functional API**](https://keras.io/getting-started/functional-api-guide/)를 사용하여 모델이 원하는 출력을 하도록 정의합니다.

모델을 정의하는 Functional API를 사용하면 단순히 입력 및 출력을 정의 할 수 있습니다.

`model = Model(inputs, outputs)`

참고: [tf.keras.applications.vgg19.VGG19()](https://keras.io/applications/#vgg19)


In [0]:
def get_model():
  """ Creates our model with access to intermediate layers. 
  
  This function will load the VGG19 model and access the intermediate layers. 
  These layers will then be used to create a new model that will take input image
  and return the outputs from these intermediate layers from the VGG model. 
  
  Returns:
    returns a keras model that takes image inputs and outputs the style and 
      content intermediate layers. 
  """
  # Load our model. We load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  # Get output layers corresponding to style and content layers 
  style_outputs = [vgg.get_layer(name).output for name in style_layers]
  content_outputs = [vgg.get_layer(name).output for name in content_layers]
  model_outputs = style_outputs + content_outputs
  # Build model 
  return models.Model(vgg.input, model_outputs)

위의 코드에서 pretrained된 이미지 분류 네트워크를 로드합니다. 그런 다음 이전에 정의한 관심있는 layer를 불러옵니다. 그런 다음 모델의 입력을 이미지로 설정하고 출력을 스타일 및 콘텐츠 레이어의 출력으로 설정하여 모델을 정의합니다. 즉, 입력 이미지를 가져와 콘텐츠 및 스타일 중간 레이어를 출력하는 모델을 만들었습니다.


## Define and create our loss functions (content and style distances)

### Content Loss

우리의 콘텐츠 loss 정의는 실제로 매우 간단합니다. 원하는 콘텐츠 이미지와 기본 입력 이미지를 네트워크에 전달합니다. 이렇게하면 모델에서 출력되는 중간 레이어 출력 (위에 정의 된 레이어에서)이 반환됩니다. 그런 다음 우리는 단순히 그 이미지들의 두 중간 representation 사이의 유클리드 거리를 취합니다.


보다 공식적으로, 콘텐츠 손실은 출력 이미지 $x$와 콘텐츠 이미지 $p$에서 콘텐츠까지의 거리를 설명하는 함수입니다. $ C_{nn} $은 미리 훈련 된 deep convolutional neural network라고 합시다. 우리는 [VGG19](https://keras.io/applications/#vgg19)를 사용할 것입니다.  $X$를 임의의 이미지라고 하면 $C_{nn}(X)$ 는 네트워크에 X를 넣은 것입니다. $F^l_{ij}(x) \in C_{nn}(x)$ 와 $P^l_{ij}(p) \in C_{nn}(p)$ 를 각각 입력으로 $x$ 와 $p$ 를 넣었을때 layer $l$ 에서의 중간 feature representation이라고 합시다. 그리면 우리는 콘텐츠 거리(loss)를 수식적으로 다음과 같이 정의 할 수 있습니다: $$L^l_{content}(p, x) = \sum_{i, j} (F^l_{ij}(x) - P^l_{ij}(p))^2$$

우리는 일반적인 방식으로 backpropagation을 수행하여 이러한 콘텐츠 loss를 최소화합니다. 따라서 특정 레이어 (content_layer에 정의 됨)에서 원본 콘텐츠 이미지와 같은 응답을 생성 할 때까지 초기 이미지를 변경합니다.

이것은 매우 간단하게 구현 될 수 있습니다. 입력 이미지 $x$, 그리고 우리의 콘텐트 이미지 $p$를 입력으로 받은 네트워크의 레이어 $l$에서 feature map을 입력으로 받아서 컨텐츠 거리를 반환합니다.


### Computing content loss
실제로 원하는 각 레이어에서 콘텐츠 loss를 추가 할 것입니다. 이 방법은 우리가 모델을 통해 입력 이미지를 공급할 때마다 (eager에서는 단순하게 `model(input_image)`입니다!) 모델을 통한 모든 컨텐츠 손실이 적절하게 계산 될 것이고 eager로 실행하기 때문에 모든 gradients가 계산됩니다 .

In [0]:
def get_content_loss(base_content, target):
  return tf.reduce_mean(tf.square(base_content - target))

### Style Loss

스타일 loss 계산은 좀 더 복잡하지만 동일한 원칙을 따르며, 이번에는 네트워크에 기본 입력 이미지와 스타일 이미지를 입력으로 줍니다. 그러나 기본 입력 이미지와 스타일 이미지의 중간 출력을 그대로 비교하는 대신 두 출력의 Gram matrix를 비교합니다.

수학적으로, 우리는 기본 입력 이미지 $x$와 스타일 이미지 $a$의 style loss를 두 이미지의 스타일 표현(gram matrix)의 거리로 정의합니다. 우리는 이미지의 스타일 표현을 gram matrix $G^l$로 주어지는 서로 다른 필터 응답의 correlation으로 설명합니다. 여기서 $G^l_{ij}$는 벡터화 된 feature map $i$와 $j$의 내적 (inner product) 입니다. 우리는 특정 이미지의 feature map에서 생성된 $G^l_{ij}$가 feature map $i$와 $j$ 사이의 correlation을 나타낸다는 것을 알 수 있습니다.

기본 입력 이미지의 스타일을 생성하기 위해 콘텐츠 이미지에서 gradient descent를 수행하여 스타일 이미지의 스타일 표현과 일치하는 이미지로 변환합니다.이를 위해  스타일 이미지와 입력 이미지 사이의 mean square 거리를 최소화하도록 만듭니다. 총 스타일 손실에 대한 각 layer의 contribution은 다음과 같습니다: 

$$E_l = \frac{1}{4N_l^2M_l^2} \sum_{i,j}(G^l_{ij} - A^l_{ij})^2$$

$G^l_{ij}$ 와 $A^l_{ij}$는 각각 layer $l$에서 
$x$ 와 $a$의 스타일 표현입니다. $N_l$는 각 사이즈가 $M_l = height * width$인 feature map 수를 나타냅니다. 따라서 전체 스타일 loss는

$$L_{style}(a, x) = \sum_{l \in L} w_l E_l$$

입니다. 여기서 우리는 각 layer의 loss contribution을 $w_l$로 가중치 주었습니다. 우리의 경우에 각 layer를 동일하게 가중치 주었습니다($w_l =\frac{1}{|L|}$).

### Total loss
만들고자하는 이미지는 콘텐츠 이미지와 $L_{content}$가 작고 스타일 이미지와 $L_{style}$이 작아지도록 하는 이미지입니다. 따라서 전체 목적 함수(loss)는 다음과 같습니다:

$$L_{total}(p, a, x) = \alpha L_{content}(p, x)+\beta L_{style}(a, x)$$

$\alpha$와 $\beta$는 각각 콘텐트와 스타일 loss에 곱해지는 weight 값 입니다.

### Computing style loss
이번에도 style loss를 거리  metric으로 구현합니다. 

get_style_loss는 $E_l$을 구하는 함수입니다.

In [0]:
def gram_matrix(input_tensor):
  # We make the image channels first 
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a=True)
  return gram / tf.cast(n, tf.float32)

def get_style_loss(base_style, gram_target):
  """Expects two images of dimension h, w, c"""
  # height, width, num filters of each layer
  # We scale the loss at a given layer by the size of the feature map and the number of filters
  height, width, channels = base_style.get_shape().as_list()
  gram_style = gram_matrix(base_style)
  
  return tf.reduce_mean(tf.square(gram_style - gram_target))# / (4. * (channels ** 2) * (width * height) ** 2)

## Apply style transfer to our images


### Run Gradient Descent 
우리는 loss를 최소화하기 위해 [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)* optimizer를 사용합니다. 반복적으로 출력 이미지를 업데이트하여 loss를 최소화합니다. 네트워크와 관련된 weight를 업데이트하지 않고 대신 입력 이미지를 조정하여 loss를 최소화합니다. 이를 위해서는 loss와 gradients를 계산하는 방법을 알아야합니다.

우리는 콘텐츠와 스타일 이미지를 불러오고, 네트워크를 통해 feed forward하며, 모델에서 콘텐츠 및 스타일 feature representation을 출력하는 작은 도우미 함수를 정의 할 것입니다.

In [0]:
def get_feature_representations(model, content_path, style_path):
  """Helper function to compute our content and style feature representations.

  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image
    
  Returns:
    returns the style features and the content features. 
  """
  # Load our images in 
  content_image = load_and_process_img(content_path)
  style_image = load_and_process_img(style_path)
  
  # batch compute content and style features
  style_outputs = model(style_image)
  content_outputs = model(content_image)
  
  
  # Get the style and content feature representations from our model  
  style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
  content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]
  return style_features, content_features

### Computing the loss and gradients
여기서는 [**tf.GradientTape**](https://www.tensorflow.org/programmers_guide/eager#computing_gradients)을 사용하여 gradient를 계산합니다. 나중에 gradient를 계산하기위한 operation을 추적하여 자동 미분화를 가능하게 합니다. Forward pass중에 작업을 기록한 다음 backward pass시에 입력 이미지에 대하여 loss 함수의 gradient를 계산할 수 있습니다.


In [0]:
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
  """This function will compute the loss total loss.
  
  Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
  Returns:
    returns the total loss, style loss, content loss, and total variational loss
  """
  style_weight, content_weight = loss_weights
  
  # Feed our init image through our model. This will give us the content and 
  # style representations at our desired layers. Since we're using eager
  # our model is callable just like any other function!
  model_outputs = model(init_image)
  
  style_output_features = model_outputs[:num_style_layers]
  content_output_features = model_outputs[num_style_layers:]
  
  style_score = 0
  content_score = 0

  # Accumulate style losses from all layers
  # Here, we equally weight each contribution of each loss layer
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, style_output_features):
    style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
    
  # Accumulate content losses from all layers 
  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, content_output_features):
    content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
  style_score *= style_weight
  content_score *= content_weight

  # Get total loss
  loss = style_score + content_score 
  return loss, style_score, content_score

Gradients를 구하는 것은 쉽습니다:

In [0]:
def compute_grads(cfg):
  with tf.GradientTape() as tape: 
    all_loss = compute_loss(**cfg)
  # Compute gradients wrt input image
  total_loss = all_loss[0]
  return tape.gradient(total_loss, cfg['init_image']), all_loss

### Optimization loop

In [0]:
import IPython.display

def run_style_transfer(content_path, 
                       style_path,
                       num_iterations=1000,
                       content_weight=1e3, 
                       style_weight=1e-2): 
  # We don't need to (or want to) train any layers of our model, so we set their
  # trainable to false. 
  model = get_model() 
  for layer in model.layers:
    layer.trainable = False
  
  # Get the style and content feature representations (from our specified intermediate layers) 
  style_features, content_features = get_feature_representations(model, content_path, style_path)
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
  
  # Set initial image
  init_image = load_and_process_img(content_path)
  init_image = tfe.Variable(init_image, dtype=tf.float32)
  # Create our optimizer
  opt = tf.train.AdamOptimizer(learning_rate=5, beta1=0.99, epsilon=1e-1)

  # For displaying intermediate images 
  iter_count = 1
  
  # Store our best result
  best_loss, best_img = float('inf'), None
  
  # Create a nice config 
  loss_weights = (style_weight, content_weight)
  cfg = {
      'model': model,
      'loss_weights': loss_weights,
      'init_image': init_image,
      'gram_style_features': gram_style_features,
      'content_features': content_features
  }
    
  # For displaying
  num_rows = 2
  num_cols = 5
  display_interval = num_iterations/(num_rows*num_cols)
  start_time = time.time()
  global_start = time.time()
  
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means   
  
  imgs = []
  for i in range(num_iterations):
    grads, all_loss = compute_grads(cfg)
    loss, style_score, content_score = all_loss
    opt.apply_gradients([(grads, init_image)])
    clipped = tf.clip_by_value(init_image, min_vals, max_vals)
    init_image.assign(clipped)
    end_time = time.time() 
    
    if loss < best_loss:
      # Update best loss and best image from total loss. 
      best_loss = loss
      best_img = deprocess_img(init_image.numpy())

    if i % display_interval== 0:
      start_time = time.time()
      
      # Use the .numpy() method to get the concrete numpy array
      plot_img = init_image.numpy()
      plot_img = deprocess_img(plot_img)
      imgs.append(plot_img)
      IPython.display.clear_output(wait=True)
      IPython.display.display_png(Image.fromarray(plot_img)) # NumPy 배열을 Image 객체로 변환
      print('Iteration: {}'.format(i))        
      print('Total loss: {:.4e}, ' 
            'style loss: {:.4e}, '
            'content loss: {:.4e}, '
            'time: {:.4f}s'.format(loss, style_score, content_score, time.time() - start_time))
  print('Total time: {:.4f}s'.format(time.time() - global_start))
  IPython.display.clear_output(wait=True)
  plt.figure(figsize=(14,4))
  for i,img in enumerate(imgs):
      plt.subplot(num_rows,num_cols,i+1)
      plt.imshow(img)
      plt.xticks([])
      plt.yticks([])
      
  return best_img, best_loss 

In [0]:
# best, best_loss = run_style_transfer(content_path, 
#                                      style_path, num_iterations=1000)

In [0]:
# Image.fromarray(best)

## Visualize outputs
우리는 출력 이미지에 적용된 processing을 제거하기 위해 출력 이미지를 "deprocess"합니다.

In [0]:
def show_results(best_img, content_path, style_path, show_large_final=True):
  plt.figure(figsize=(10, 5))
  content = load_img(content_path) 
  style = load_img(style_path)

  plt.subplot(1, 2, 1)
  imshow(content, 'Content Image')

  plt.subplot(1, 2, 2)
  imshow(style, 'Style Image')

  if show_large_final: 
    plt.figure(figsize=(10, 10))

    plt.imshow(best_img)
    plt.title('Output Image')
    plt.show()

In [0]:
# show_results(best, content_path, style_path)

## Try it on other images
Image of Tuebingen 

Photo By: Andreas Praefcke [GFDL (http://www.gnu.org/copyleft/fdl.html) or CC BY 3.0  (https://creativecommons.org/licenses/by/3.0)], from Wikimedia Commons

### Starry night + Tuebingen

In [0]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [0]:
show_results(best_starrㅁy_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

### Pillars of Creation + Tuebingen

In [0]:
best_poc_tubingen, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg', 
                                                  '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

In [0]:
show_results(best_poc_tubingen, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

### Kandinsky Composition 7 + Tuebingen

In [0]:
best_kandinsky_tubingen, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg', 
                                                  '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

In [0]:
show_results(best_kandinsky_tubingen, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

### Pillars of Creation + Sea Turtle

In [0]:
best_poc_turtle, best_loss = run_style_transfer('/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg', 
                                                  '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

In [0]:
show_results(best_poc_turtle, 
             '/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg',
             '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

## 주요 요점

### 다룬 내용들:

* 우리는 몇 가지 다른 loss 함수를 구축하고 이러한 손실을 최소화도록 입력 영상을 변환하기 위해 backpropagation 사용했습니다
  * 이를 위해 **pretrained된 모델**을 로드하고 학습 된 feature map을 사용하여 이미지의 콘텐츠 및 스타일 표현을 설명해야했습니다.
    * 우리의 주요 loss 함수는 주로 이러한 다양한 representation의 관점에서 거리를 계산하는 것이 었습니다
* 우리는 이것을 custom model과 **eager execution**으로 구현하였습니다.
* 우리는 Functional API를 이용하여 우리의 custom model을 만들었습니다.
  * Eager execution은 자연스러운 Python control flow를 사용하여 텐서로 동적으로 작업 할 수있게 해줍니다.
  * 우리는 텐서를 직접 조작하여 디버깅을하고 텐서로 작업하는 것을 더 쉽게 만듭니다.
* **tf.gradient**를 사용하여 optimizer 업데이트 규칙을 적용하고 이미지를 반복적으로 업데이트했습니다. Optimizer는 입력 이미지와 관련하여 주어진 loss를 최소화했습니다.


**[Image of Tuebingen](https://commons.wikimedia.org/wiki/File:Tuebingen_Neckarfront.jpg)** 
Photo By: Andreas Praefcke [GFDL (http://www.gnu.org/copyleft/fdl.html) or CC BY 3.0  (https://creativecommons.org/licenses/by/3.0)], from Wikimedia Commons

**[Image of Green Sea Turtle](https://commons.wikimedia.org/wiki/File:Green_Sea_Turtle_grazing_seagrass.jpg)**
By P.Lindgren [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], from Wikimedia Commons



# Report

1. 튀빙겐 사진을 고흐의 starry night 스타일로 바꿔봅시다. content_weight=1e3, style_weight=1e-2
 
2. 튀빙겐 사진을 고흐의 starry night 스타일로 바꿔봅시다. content_weight=1e3, style_weight=1e-0

3. 튀빙겐 사진을 고흐의 starry night 스타일로 바꿔봅시다. content_weight=1e3, style_weight=1e--4

4. 튀빙겐 사진을 고흐의 starry night 스타일로 바꿔봅시다. content_weight=1e1, style_weight=1e-2
 
5. 튀빙겐 사진을 고흐의 starry night 스타일로 바꿔봅시다. content_weight=1e5, style_weight=1e-2

Q) $\alpha$(content_weight)와 $\beta$(style_weight)의 역할은 무엇입니까?

In [0]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg', content_weight=1e3, style_weight=1e-2)

show_results(best_starry_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
              '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [0]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg', content_weight=1e3, style_weight=1e-0)

show_results(best_starry_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
              '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [0]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg', content_weight=1e3, style_weight=1e-4)

show_results(best_starry_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
              '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [0]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg', content_weight=1e1, style_weight=1e-2)

show_results(best_starry_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
              '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [0]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg', content_weight=1e5, style_weight=1e-2)

show_results(best_starry_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
              '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

A) 각각 이미지를 만들 때 content 정보와  style 정보의 가중치를 얼마나 줄지를 나타내며, 두 값의 비율로 콘텐츠와 스타일의 비율을 조절할 수 있다. 